In [90]:
! cd ../data/retrotech && head signals.csv

import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from aips import *
from session_gen import SessionGenerator
import os
from IPython.core.display import display,HTML

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"


In [91]:
def all_sessions():
    import glob
    return pd.concat([pd.read_csv(f, compression='gzip')
                      for f in glob.glob('*_sessions.gz')])

sessions = all_sessions()
sessions

,sess_id,query,rank,clicked_doc_id,clicked
0,2,dryer,0.0,12505451713,False
1,2,dryer,1.0,84691226727,True
2,2,dryer,2.0,883049066905,False
3,2,dryer,3.0,14381196320,False
4,2,dryer,4.0,74108056764,False
...,...,...,...,...,...
149995,55001,blue ray,25.0,22265004517,False
149996,55001,blue ray,26.0,885170038875,False
149997,55001,blue ray,27.0,786936817232,False
149998,55001,blue ray,28.0,600603132872,False


In [92]:
def fetch_products(doc_ids):
    import requests
    doc_ids = ["%s" % doc_id for doc_id in doc_ids]
    query = "upc:( " + " OR ".join(doc_ids) + " )"
    params = {'q':  query, 'wt': 'json', 'rows': len(doc_ids)}
    resp = requests.get('http://aips-solr:8983/solr/products/select', params=params)
    df = pd.DataFrame(resp.json()['response']['docs'])
    df['upc'] = df['upc'].astype('int64')
    
    df.insert(0, 'image', df.apply(lambda row: "<img height=\"100\" src=\"" + img_path_for_upc(row['upc']) + "\">", axis=1))
    
    return df

def render(judged, judgments_col='ctr', label=""):
    """ Render the computed judgments alongside the productns and description data"""
    w_prods = judged.merge(products, left_on='clicked_doc_id', right_on='upc', how='left')

    w_prods = w_prods[[judgments_col, 'image', 'upc', 'name', 'shortDescription']]

    return HTML(f"<h1>{label}</h1>" + w_prods.to_html(escape=False))

products = fetch_products(doc_ids=sessions['clicked_doc_id'].unique())

In [93]:
QUERY='dryer'

In [94]:
#dryer_sessions = dryer_sessions.merge(dryer_products, left_on='clicked_doc_id', right_on='upc', how='left')
dryer_sessions=sessions[sessions['query'] == QUERY]

num_sessions = len(dryer_sessions['sess_id'].unique())
dryer_ctrs = dryer_sessions.groupby('clicked_doc_id')['clicked'].sum().reset_index()
dryer_ctrs['ctr'] = dryer_ctrs['clicked'] / num_sessions
#dryer_ctrs = dryer_ctrs.sort_values('clicked', ascending=False)\
#                       .merge(products, left_on='clicked_doc_id', right_on='upc', how='left')
#dryer_sessions.groupby('clicked_doc_id').sum().sort_values('clicked', ascending=False)

render(dryer_ctrs.sort_values('ctr', ascending=False), judgments_col='ctr', label="Click-Thru-Rate Judgments")

,ctr,image,upc,name,shortDescription
0,0.1608,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
1,0.0816,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
2,0.0710,,12505451713,Frigidaire - Semi-Rigid Dryer Vent Kit - Silver,Expandable vent; custom fitted ends and clamps
3,0.0576,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
4,0.0572,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
5,0.0552,,77283045400,Hello Kitty - Hair Dryer - Pink,1875 watts of power; high and low heat settings; cool shot button; detachable styling nozzle
6,0.0546,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
7,0.0540,,665331101927,Everything in Static - CD,\N
8,0.0536,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
9,0.0470,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser


## A typical session

What's a typical search session look like? Compute average rank per each product

In [95]:
dryer_sessions=sessions[sessions['query'] == QUERY]

typical_page = dryer_sessions.groupby('clicked_doc_id')['rank'].mean().reset_index().sort_values('rank')

render(typical_page, judgments_col='rank', label="A Typical Results Page")

,rank,image,upc,name,shortDescription
0,1.9124,,12505451713,Frigidaire - Semi-Rigid Dryer Vent Kit - Silver,Expandable vent; custom fitted ends and clamps
1,2.8290,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
2,3.5726,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
3,4.4552,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
4,5.1276,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
5,5.9318,,77283045400,Hello Kitty - Hair Dryer - Pink,1875 watts of power; high and low heat settings; cool shot button; detachable styling nozzle
6,6.7610,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
7,7.5138,,665331101927,Everything in Static - CD,\N
8,8.3308,,14381196320,The Mind Snatchers - DVD,\N
9,9.1230,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser


In [96]:
num_sessions = len(sessions['sess_id'].unique())
global_ctrs = sessions.groupby('rank')['clicked'].sum() / num_sessions
global_ctrs = global_ctrs.reset_index().rename(columns={'clicked': 'global_ctr'})
global_ctrs

,rank,global_ctr
0,0.0,0.235600
1,1.0,0.139655
2,2.0,0.080782
3,3.0,0.059455
4,4.0,0.054327
5,5.0,0.040673
6,6.0,0.031582
7,7.0,0.032709
8,8.0,0.020145
9,9.0,0.017800


In [97]:
#coec
coec_dryer_sums = dryer_sessions.groupby(['query', 'clicked_doc_id', 'rank'])['clicked'].sum()
coec_dryer_counts = dryer_sessions.groupby(['query', 'clicked_doc_id', 'rank'])['clicked'].count()

coecs = coec_dryer_sums / coec_dryer_counts
coecs = coecs.reset_index().rename(columns={'clicked': 'ctr_per_posn'})

coecs = coecs.merge(global_ctrs, on='rank', how='left')
coecs['ctr_over_global'] = coecs['ctr_per_posn'] / coecs['global_ctr']

coecs = coecs.groupby(['query', 'clicked_doc_id'])['ctr_over_global'].mean().reset_index()\
             .sort_values('ctr_over_global', ascending=False);

render(coecs, judgments_col='ctr_over_global', label="Clicks Over Expected Clicks")

,ctr_over_global,image,upc,name,shortDescription
0,2.656086,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
1,2.417052,,665331101927,Everything in Static - CD,\N
2,2.234264,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
3,2.180631,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
4,1.971325,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
5,1.963238,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
6,1.905567,,36725578241,Samsung - 7.3 Cu. Ft. 7-Cycle Electric Dryer - White,Soft-touch dial controls; 7 preset drying cycles; 4 temperature settings; powdercoat drum; noise reduction package
7,1.853835,,14381196320,The Mind Snatchers - DVD,\N
8,1.828587,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
9,1.775106,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation


In [98]:
#sdbn
# For each session, find last click
# - everything above last click considered examined
# - everything clicked counted towards attractive
# - satisfaction is clicks / last clicks

last_clicks = dryer_sessions[dryer_sessions['clicked']].groupby(['sess_id', 'clicked'])['rank'].max().reset_index()
last_clicks = last_clicks.rename(columns={'rank': 'last_click_rank'})[['sess_id','last_click_rank']]
                            
# PROBLEM eliminate sessions without clicks!
sdbn_sessions = dryer_sessions.merge(last_clicks, on='sess_id', how='left')

sdbn_sessions['satisfied'] = sdbn_sessions.apply(lambda row: row['rank'] == row['last_click_rank'], axis=1)
sdbn_sessions['attractive'] = sdbn_sessions.apply(lambda row: row['rank'] <= row['last_click_rank'] and row['clicked'], axis=1)
sdbn_sessions['examined'] = sdbn_sessions.apply(lambda row: row['rank'] <= row['last_click_rank'], axis=1)



#    df.insert(0, 'image', df.apply(lambda row: "<img height=\"100\" src=\"" + img_path_for_upc(row['upc']) + "\">", axis=1))

In [99]:
# Clicks over examines
sdbn = sdbn_sessions[sdbn_sessions['examined']].groupby('clicked_doc_id').sum()

sdbn['attractiveness'] = sdbn['attractive'] / sdbn['examined']
sdbn['satisfaction'] = sdbn['satisfied'] / sdbn['examined']

sdbn = sdbn.sort_values('attractiveness', ascending=False)

render(sdbn, judgments_col='attractiveness', label="Attractive Results")

,attractiveness,image,upc,name,shortDescription
0,0.411765,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
1,0.392435,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
2,0.316411,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
3,0.293785,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
4,0.275154,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
5,0.272537,,36725578241,Samsung - 7.3 Cu. Ft. 7-Cycle Electric Dryer - White,Soft-touch dial controls; 7 preset drying cycles; 4 temperature settings; powdercoat drum; noise reduction package
6,0.260391,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
7,0.257009,,12505527456,"Smart Choice - 1/2"" Safety+PLUS Stainless-Steel Gas Dryer Connector","Safety+PLUS automatic shut-off valve; leak detection solution; pipe thread sealant; 60,500 BTU; CSA approved"
8,0.214221,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
9,0.208042,,36725561977,Samsung - 3.5 Cu. Ft. 6-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedSoft dial touch pad controls; 6 cycles; delay-start; child lock; Vibration Reduction Technology


In [100]:
sdbn = sdbn.sort_values('satisfaction', ascending=False)

render(sdbn, judgments_col='satisfaction', label="Satisfying? Results")

,satisfaction,image,upc,name,shortDescription
0,0.321981,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
1,0.278960,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
2,0.248588,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
3,0.219713,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
4,0.213836,,36725578241,Samsung - 7.3 Cu. Ft. 7-Cycle Electric Dryer - White,Soft-touch dial controls; 7 preset drying cycles; 4 temperature settings; powdercoat drum; noise reduction package
5,0.207944,,12505527456,"Smart Choice - 1/2"" Safety+PLUS Stainless-Steel Gas Dryer Connector","Safety+PLUS automatic shut-off valve; leak detection solution; pipe thread sealant; 60,500 BTU; CSA approved"
6,0.193154,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
7,0.170405,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
8,0.169580,,36725561977,Samsung - 3.5 Cu. Ft. 6-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedSoft dial touch pad controls; 6 cycles; delay-start; child lock; Vibration Reduction Technology
9,0.155880,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
